In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy
import matplotlib.pyplot as plt
import kalepy as kale


import matplotlib.cm as cm
from holodeck import plot, utils
from holodeck.constants import YR, MSOL
import holodeck.detstats as det

# Check ss_lib_combine is working

In [ ]:
import h5py

In [ ]:
ssfile = h5py.File('/Users/emigardiner/GWs/holodeck/output/awg_output/uniform-07a_new_n500_r100_f40/sam_lib.hdf5', 'r+')
list(ssfile.keys())

In [ ]:
hc_bg_com = ssfile['hc_bg'][...]
print(hc_bg_com.shape)
hc_ss_com = ssfile['hc_ss'][...]
print(hc_ss_com.shape)
fobs_com = ssfile['fobs'][:]
print(fobs_com.shape)
sspar_com = ssfile['sspar'][...]
print(sspar_com.shape)
bgpar_com = ssfile['bgpar'][...]
print(bgpar_com.shape)


In [ ]:
nsort, fidx, hc_ref = holo.detstats.rank_samples(hc_ss_com, hc_bg_com, fobs_com, fidx=1, hc_ref=holo.detstats.HC_REF15_10YR, ret_all=True)
np.savez('/Users/emigardiner/GWs/holodeck/output/awg_output/uniform-07a_new_n500_r100_f40/sample_rankinsg.npz', nsort)
print(nsort[:10])
print(fobs_com[fidx]*YR)
print(hc_ref)

In [ ]:
for ii in range(0,1): #len(hc_bg_com)):
    fig = plot.plot_bg_ss(fobs_com[:], bg=hc_bg_com[ii], ss=hc_ss_com[ii])
    fig = plot.plot_pars(fobs_com[:], hc_ss_com[ii,...], hc_bg_com[ii,...], sspar_com[ii,...], bgpar_com[ii,...])


## without pars

In [ ]:
# ssfile = h5py.File('/Users/emigardiner/GWs/holodeck/output/2023-05-03-mbp-ss05_n10_r10_f100_d17.5_l100_p1_s10/ss_lib.hdf5', 'r+')
# list(ssfile.keys())

In [ ]:
# hc_bg_com2 = ssfile['hc_bg']
# print(hc_bg_com2.shape)
# hc_ss_com2 = ssfile['hc_ss']
# print(hc_ss_com2.shape)
# fobs_com2 = ssfile['fobs']
# print(fobs_com2.shape)
# # sspar_com = ssfile['sspar']
# # bgpar_com = ssfile['bgpar']


In [ ]:
# for ii in range(len(hc_bg_com2)):
#     fig = plot.plot_gwb(fobs_com2[:], gwb=hc_bg_com2[ii], hc_ss=hc_ss_com2[ii,:,:,0:2])
#     # fig = plot.plot_pars(fobs_com[:], hc_ss_com[ii,...,0:2], hc_bg_com[ii,...,0:2], sspar_com[ii,...,0:2], bgpar_com[ii,...,0:2])


# Check npz arrays

In [ ]:
filename = '/Users/emigardiner/GWs/holodeck/brc_output/lib_ss__p000005_ss42.npz'
infile = np.load(filename)
print(infile.files)
fobs_npz = infile['fobs']
xx = fobs_npz*YR
hc_ss_npz = infile['hc_ss']
print(hc_ss_npz.shape)
hc_bg_npz = infile['hc_bg']
print(hc_bg_npz.shape)
sspar_npz = infile['sspar']
print(sspar_npz.shape)
bgpar_npz = infile['bgpar']
print(bgpar_npz.shape)

really slow to plot for 1000 loudest

In [ ]:
fig1 = plot.plot_bg_ss(fobs_npz, bg=hc_bg_npz, ss=hc_ss_npz[...,0:15])

In [ ]:
fig2 = plot.plot_pars(fobs_npz, hc_ss_npz[...,:100], hc_bg_npz, sspar_npz[...,:100], bgpar_npz)

trying all 1000

In [ ]:
fig1 = plot.plot_bg_ss(fobs_npz, bg=hc_bg_npz, ss=hc_ss_npz)

In [ ]:
fig2 = plot.plot_pars(fobs_npz, hc_ss_npz, hc_bg_npz, sspar_npz, bgpar_npz)

# Check 100 loudest

In [ ]:
dur = 17.5*YR
cad = 1/12*YR
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model() 
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version
hard = holo.hardening.Hard_GW

Calculate strain

In [ ]:
fobs_orb_edges = fobs_edges / 2.0 
fobs_orb_cents = fobs/ 2.0
# edges
edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents) # should the zero stalled option be part of the parameter space?
edges[-1] = fobs_orb_edges
# integrate for number
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_edges)) 
hc_ss, hc_bg, sspar, bgpar = ss.ss_gws(edges, number, realize=30, \
                                        loudest = 100, params = True) 

In [ ]:
print(fobs.shape)

In [ ]:
fig = plot.plot_gwb(fobs, gwb=hc_bg, hc_ss=hc_ss)

In [ ]:
fig = plot.plot_pars(fobs, hc_ss, hc_bg, sspar, bgpar)

# Check 1000 loudest

calculate strain, same SAM

In [ ]:
fobs_orb_edges = fobs_edges / 2.0 
fobs_orb_cents = fobs/ 2.0
# edges
edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents) # should the zero stalled option be part of the parameter space?
edges[-1] = fobs_orb_edges
# integrate for number
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_edges)) 
hc_ss1000, hc_bg1000, sspar1000, bgpar1000 = ss.ss_gws(edges, number, realize=30, \
                                        loudest = 1000, params = True) 

In [ ]:
fig = plot.plot_gwb(fobs, gwb=hc_bg1000, hc_ss=hc_ss1000)

In [ ]:
fig = plot.plot_pars(fobs, hc_ss1000, hc_bg1000, sspar1000, bgpar1000)